In [1]:
import torch
torch.set_default_tensor_type(torch.FloatTensor)
from torch.autograd import Variable
from torch.nn import functional as F
from torchmetrics import Accuracy, Recall, Precision, Specificity, ConfusionMatrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_recall_curve, average_precision_score,roc_curve, auc, precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt
from collections import Counter
import random
import math
import torch.optim as optim
from tabulate import tabulate
from ray import tune

from fractions import Fraction

import ray

In [2]:
# Create Tensors to hold input and outputs.
#1,3,1+1,2,1
#第一组参数
# beta_1=np.append(np.array([4,4,-8,-4,6,8,-4,4,-6,-8]),np.zeros(90))
# beta_2=np.append(np.array([-2,2,4,-2,3,-4,2,-2,-3,4]),np.zeros(90))
# beta_3=np.append(np.array([-3,-3,6,-3,-4.5,-6,3,-3,4.5,6]),np.zeros(90))

#第二组参数

beta_1=np.append(np.array([4,4,-8,-4,6,-4,8,4,-6,-8]),np.zeros(90))
beta_2=np.append(np.array([-2,2,4,-2,3,2,-4,-2,-3,4]),np.zeros(90))
beta_3=np.append(np.array([-3,-3,6,-3,-4.5,3,-6,-3,4.5,6]),np.zeros(90))

beta_1=beta_1.reshape(-1,1)
beta_2=beta_2.reshape(-1,1)
beta_3=beta_3.reshape(-1,1)

n1=200
n2=200
n3=200
p=100
corval=0
prior=[11,12,13,14,15]

def sigmoid(X):
    return .5 * (1 + np.tanh(.5 * X))

def generate_data(corval,beta,n,p):
    mean=np.zeros(p)
    sigma=np.array([[corval**abs(i-j) for i in range(p)] for j in range(p)])
    x=torch.tensor(np.random.multivariate_normal(mean=mean,cov=sigma,size=n))
    Pi_test= sigmoid(x@beta)
    y=np.random.binomial(1,Pi_test.ravel(),n)
    x=x.to(torch.float32)
    #y=torch.tensor(y,dtype=torch.double)
    y=torch.tensor(y,dtype=torch.float)
    #y=y.to(torch.float32)
    return x,y
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

def cubic_root(x):
    return math.copysign(math.pow(abs(x), 1.0/3.0), x)

In [3]:
def train_model(config):
    split_num=5
    kf = KFold(n_splits=split_num)



    sum_test_BCE_loss=0
    for idx_train,idx_test in kf.split(inputs_1):
        lambda1,lambda2,lr,ga= config["lambda1"], config["lambda2"],config["lr"],config["ga"]
        model_1=MLP(seed=1)
        model_2=MLP(seed=2)
        model_3=MLP(seed=1)
        max_iteration=1000
        learning_rate=lr
        params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
        optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
        loss_fn = torch.nn.BCELoss()
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=ga)

        BCE=[]

        for t in range(max_iteration):
            optimizer.zero_grad() # renew optimizer
            out_1, layer1_out_1, layer2_out_1= model_1(inputs_1[idx_train])
            out_2, layer1_out_2, layer2_out_2= model_2(inputs_2[idx_train])
            out_3, layer1_out_3, layer2_out_3= model_3(inputs_3[idx_train])# forward propagate

            # extract parameters
            #[:-1] for leaving out bias term#
            model_1_all_linear1_params = model_1.linear1
            model_1_all_linear2_params= torch.cat([x.view(-1) for x in model_1.linear2.parameters()][:-1])
            model_1_all_linear3_params= torch.cat([x.view(-1) for x in model_1.linear3.parameters()][:-1])
           

            model_2_all_linear1_params = model_1.linear1
            model_2_all_linear2_params= torch.cat([x.view(-1) for x in model_2.linear2.parameters()][:-1])
            model_2_all_linear3_params= torch.cat([x.view(-1) for x in model_2.linear3.parameters()][:-1])
    

            model_3_all_linear1_params = model_3.linear1
            model_3_all_linear2_params= torch.cat([x.view(-1) for x in model_3.linear2.parameters()][:-1])
            model_3_all_linear3_params= torch.cat([x.view(-1) for x in model_3.linear3.parameters()][:-1])
    


            # compute loss
            all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)
            all_linear23_params=torch.cat((model_1_all_linear2_params,model_1_all_linear3_params,
                                           model_2_all_linear2_params,model_2_all_linear3_params,
                                           model_3_all_linear2_params,model_3_all_linear3_params),0)

            non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)

            BCE_loss_1 = loss_fn(out_1, targets_1[idx_train].reshape(-1,1))
            BCE_loss_2 = loss_fn(out_2, targets_2[idx_train].reshape(-1,1))
            BCE_loss_3 = loss_fn(out_3, targets_3[idx_train].reshape(-1,1))

#             l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)
            l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

            l2_regularization = lambda2* torch.sum(all_linear23_params.pow(2))
            BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
            loss =BCE_loss+l1_regularization +l2_regularization

            # record loss
            BCE.append(BCE_loss.item())

            # compute derivative
            loss.backward()

            # gradient descent
            optimizer.step()

            # learning rate decay
            scheduler.step()
        
        
        test_out_1, test_layer1_out_1, test_layer2_out_1= model_1(train_x1[idx_test])
        test_out_2, test_layer1_out_2, test_layer2_out_2= model_2(train_x2[idx_test])
        test_out_3, test_layer1_out_3, test_layer2_out_3= model_3(train_x3[idx_test])
        
        test_BCE_loss_1 = loss_fn(test_out_1, targets_1[idx_test].reshape(-1,1))
        test_BCE_loss_2 = loss_fn(test_out_2, targets_2[idx_test].reshape(-1,1))
        test_BCE_loss_3 = loss_fn(test_out_3, targets_3[idx_test].reshape(-1,1))
        
        test_BCE_loss=test_BCE_loss_1+test_BCE_loss_2+test_BCE_loss_3
        
        sum_test_BCE_loss+=test_BCE_loss.item()

    #print("loss: ",)
    tune.report(my_test_BCE_loss=sum_test_BCE_loss/split_num)

In [4]:
def train_model_2(config):
    split_num=5
    kf = KFold(n_splits=split_num)
    
    
    lambda1,lambda2,eta,lr= config["lambda1"], config["lambda2"],config["eta"],config["lr"]
    inputs_1,inputs_2,inputs_3=train_x1,train_x2,train_x3
    targets_1=(1-eta)*train_y1+eta*y_prior_1
    targets_2=(1-eta)*train_y2+eta*y_prior_2
    targets_3=(1-eta)*train_y3+eta*y_prior_3
    
    
    sum_tGM=0
    for idx_train,idx_test in kf.split(inputs_1):
       
        
        model_1=MLP(seed=3)
        model_2=MLP(seed=2)
        model_3=MLP(seed=1)
        max_iteration=1000
        learning_rate=lr
        params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
        optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
        loss_fn = torch.nn.BCELoss()
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

        BCE=[]

        for t in range(max_iteration):
            optimizer.zero_grad() # renew optimizer
            out_1, layer1_out_1, layer2_out_1,layer3_out_1= model_1(inputs_1[idx_train])
            out_2, layer1_out_2, layer2_out_2,layer3_out_2= model_2(inputs_2[idx_train])
            out_3, layer1_out_3, layer2_out_3,layer3_out_3= model_3(inputs_3[idx_train])# forward propagate

            # extract parameters
            #[:-1] for leaving out bias term#
            model_1_all_linear1_params = model_1.linear1
            model_1_all_linear2_params= torch.cat([x.view(-1) for x in model_1.linear2.parameters()][:-1])
            model_1_all_linear3_params= torch.cat([x.view(-1) for x in model_1.linear3.parameters()][:-1])
            model_1_all_linear4_params= torch.cat([x.view(-1) for x in model_1.linear4.parameters()][:-1])
           

            model_2_all_linear1_params = model_1.linear1
            model_2_all_linear2_params= torch.cat([x.view(-1) for x in model_2.linear2.parameters()][:-1])
            model_2_all_linear3_params= torch.cat([x.view(-1) for x in model_2.linear3.parameters()][:-1])
            model_2_all_linear4_params= torch.cat([x.view(-1) for x in model_2.linear4.parameters()][:-1])
    

            model_3_all_linear1_params = model_3.linear1
            model_3_all_linear2_params= torch.cat([x.view(-1) for x in model_3.linear2.parameters()][:-1])
            model_3_all_linear3_params= torch.cat([x.view(-1) for x in model_3.linear3.parameters()][:-1])
            model_3_all_linear4_params= torch.cat([x.view(-1) for x in model_3.linear4.parameters()][:-1])
    


            # compute loss
            all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)
            all_linear234_params=torch.cat((model_1_all_linear2_params,model_1_all_linear3_params, model_1_all_linear4_params,
                                             model_2_all_linear2_params,model_2_all_linear3_params, model_2_all_linear4_params,
                                             model_3_all_linear2_params,model_3_all_linear3_params,model_3_all_linear4_params),0)

            non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)

            BCE_loss_1 = loss_fn(out_1, targets_1[idx_train].reshape(-1,1))
            BCE_loss_2 = loss_fn(out_2, targets_2[idx_train].reshape(-1,1))
            BCE_loss_3 = loss_fn(out_3, targets_3[idx_train].reshape(-1,1))

#             l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)
            l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

            l2_regularization = lambda2* torch.sum(all_linear234_params.pow(2))
            BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
            loss =BCE_loss+l1_regularization +l2_regularization

            # record loss
            BCE.append(BCE_loss.item())

            # compute derivative
            loss.backward()

            # gradient descent
            optimizer.step()

            # learning rate decay
            scheduler.step()
        
        
        test_out_1, test_layer1_out_1, test_layer2_out_1,test_layer3_out_1= model_1(train_x1[idx_test])
        test_out_2, test_layer1_out_2, test_layer2_out_2,test_layer3_out_2= model_2(train_x2[idx_test])
        test_out_3, test_layer1_out_3, test_layer2_out_3,test_layer3_out_3= model_3(train_x3[idx_test])
        

        test_prediction_1= (model_1(train_x1[idx_test])[0]>0.5).clone().int()
        test_target_1= train_y1[idx_test].reshape(-1,1).int()

        test_prediction_2= (model_2(train_x2[idx_test])[0]>0.5).clone().int()
        test_target_2= train_y2[idx_test].reshape(-1,1).int()

        test_prediction_3= (model_3(train_x3[idx_test])[0]>0.5).clone().int()
        test_target_3= train_y3[idx_test].reshape(-1,1).int()

        test_prediction=np.append(test_prediction_1,test_prediction_2)
        test_prediction=np.append(test_prediction,test_prediction_3)
        test_prediction=test_prediction.tolist()

        test_target=np.append(test_target_1,test_target_2)
        test_target=np.append(test_target,test_target_3)
        test_target=test_target.tolist()

        final_matrix=confusion_matrix(test_target,test_prediction)
        tTPR=recall_score(test_target,test_target)#TPR
        tTNR=final_matrix[0,0]/(sum(final_matrix[0,:]))#TNR
        tGM=math.sqrt(tTPR*tTNR)

        sum_tGM+=tGM

    tune.report(my_test_sum_tGM=sum_tGM/split_num)

In [5]:
TPR=[]
TNR=[]
GM=[]
accuracy=[]
precision=[]
f1=[]=[]
vsSEN=[]
vsSPE=[]
vsGM=[]
vsMR=[]
vsCCR=[]


vs = np.zeros((1,100))

In [ ]:
for i in range(10):
    setup_seed(i)
    x1,y1=generate_data(corval,beta_1,n1,p)
    x2,y2=generate_data(corval,beta_2,n2,p)
    x3,y3=generate_data(corval,beta_3,n3,p)

    train_x1, test_x1, train_y1, test_y1= train_test_split(x1,y1, test_size=0.2, random_state=1)
    train_x2, test_x2, train_y2, test_y2= train_test_split(x2,y2, test_size=0.2, random_state=1)
    train_x3, test_x3, train_y3, test_y3= train_test_split(x3,y3, test_size=0.2, random_state=1)
    
     # switch training set
    inputs_1,targets_1 = train_x1,train_y1
    inputs_2,targets_2 = train_x2,train_y2
    inputs_3,targets_3 = train_x3,train_y3

    # switching testing set
    test_inputs_1, test_targets_1 = test_x1,test_y1
    test_inputs_2, test_targets_2 = test_x2,test_y2
    test_inputs_3, test_targets_3 = test_x3,test_y3
    
################## first step######################################  
    prior=[11,12,13,14,15]

# choose lambda1,lambda2 by ray tune
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1 = torch.nn.Parameter(torch.randn(p))
            self.linear2 = torch.nn.Linear(p,10)
            self.linear3 = torch.nn.Linear(10,1)

        def forward(self, x):
            layer1_out = self.linear1*x
            layer2_out = F.relu(self.linear2(layer1_out))
            out= torch.sigmoid(self.linear3(layer2_out))
            return out, layer1_out, layer2_out


    config={
            "lr":tune.choice([0.6,0.7,0.8]),
            "lambda1": tune.choice([0.03,0.04,0.05]),
            "lambda2": tune.choice([0.001,0.002,0.003]),
            "ga":tune.choice([0.3,0.4,0.5])
        }

    result = tune.run(
            train_model,  
            config=config,
            num_samples=30)
    
################## second step########################################    

    final_lambda=result.get_best_config('my_test_BCE_loss',mode='min')
    lambda1,lambda2,lr,ga=final_lambda["lambda1"],final_lambda["lambda2"],final_lambda["lr"],final_lambda["ga"]
    learning_rate=lr
    max_iteration=1000
    
    BCE=[]
    
#     model_1=MLP(seed=6)
#     model_2=MLP(seed=3)
#     model_3=MLP(seed=2)

    
    model_1=MLP(seed=1)
    model_2=MLP(seed=2)
    model_3=MLP(seed=1)

    loss_fn = torch.nn.BCELoss()
    params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
    optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
    
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=ga)

    loss_record=[]
    
    for t in range(max_iteration):
    
        # renew optimizer
        optimizer.zero_grad()

        # forward propagate
        out_1, layer1_out_1, layer2_out_1= model_1(inputs_1)
        out_2, layer1_out_2, layer2_out_2= model_2(inputs_2)
        out_3, layer1_out_3, layer2_out_3= model_3(inputs_3)



        # extract parameters
        #[:-1] for leaving out bias term#
        model_1_all_linear1_params = model_1.linear1
        model_1_all_linear2_params= torch.cat([x.view(-1) for x in model_1.linear2.parameters()][:-1])
        model_1_all_linear3_params= torch.cat([x.view(-1) for x in model_1.linear3.parameters()][:-1])


        model_2_all_linear1_params = model_1.linear1
        model_2_all_linear2_params= torch.cat([x.view(-1) for x in model_2.linear2.parameters()][:-1])
        model_2_all_linear3_params= torch.cat([x.view(-1) for x in model_2.linear3.parameters()][:-1])


        model_3_all_linear1_params = model_3.linear1
        model_3_all_linear2_params= torch.cat([x.view(-1) for x in model_3.linear2.parameters()][:-1])
        model_3_all_linear3_params= torch.cat([x.view(-1) for x in model_3.linear3.parameters()][:-1])


        # compute loss

        all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)
        all_linear23_params=torch.cat((model_1_all_linear2_params,model_1_all_linear3_params, 
                                         model_2_all_linear2_params,model_2_all_linear3_params,
                                         model_3_all_linear2_params,model_3_all_linear3_params),0)

        BCE_loss_1 = loss_fn(out_1, targets_1.reshape(-1,1))
        BCE_loss_2 = loss_fn(out_2, targets_2.reshape(-1,1))
        BCE_loss_3 = loss_fn(out_3, targets_3.reshape(-1,1))
        
        non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
    #     l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)

        l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

        l2_regularization = lambda2 * torch.sum(all_linear23_params.pow(2))
        BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
        loss =BCE_loss+l1_regularization +l2_regularization

        loss_record.append(loss.item())
        # record loss
        BCE.append(BCE_loss.item())

        # compute derivative
        loss.backward()

        # gradient descent
        optimizer.step()

        # learning rate decay
        scheduler.step()
    
################## Y_prior##################    
    y_prior_1= (model_1(train_x1)[0]).detach().squeeze()

    y_prior_2= (model_2(train_x2)[0]).detach().squeeze()

    y_prior_3= (model_3(train_x3)[0]).detach().squeeze()
    
    
################## third step##################

    #no prior
    prior=[]
    # choose lambda1,lambda2 by ray tune
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1 = torch.nn.Parameter(torch.randn(p))
            self.linear2 = torch.nn.Linear(p,10)
            self.linear3 = torch.nn.Linear(10,10)
            self.linear4 = torch.nn.Linear(10,1)
        def forward(self, x):
            layer1_out = self.linear1*x
            layer2_out = F.relu(self.linear2(layer1_out))
            layer3_out = F.relu(self.linear3(layer2_out))
            out= torch.sigmoid(self.linear4(layer3_out))
            return out, layer1_out, layer2_out,layer3_out

    config={
        "lambda1": tune.choice([0.03,0.04,0.05]),
        "lambda2": tune.choice([0.001,0.002,0.003]),
         "eta": tune.choice([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]),
         "lr":tune.choice([0.6,0.7,0.8])
                            }

    result = tune.run(
            train_model_2,  
            config=config,
            num_samples=280)
    

################## fourth step##################

    #no prior
    prior=[]
    #0.4,0.4
    # set hyperparamters
    final_parameter=result.get_best_config('my_test_sum_tGM',mode='max')
    lambda1,lambda2,eta,lr=final_parameter["lambda1"],final_parameter["lambda2"],final_parameter["eta"],final_parameter["lr"]
    learning_rate=lr
    max_iteration=1000

    inputs_1,inputs_2,inputs_3=train_x1,train_x2,train_x3
    targets_1=(1-eta)*train_y1+eta*y_prior_1
    targets_2=(1-eta)*train_y2+eta*y_prior_2
    targets_3=(1-eta)*train_y3+eta*y_prior_3

    # switching testing set
    test_inputs_1, test_targets_1 = test_x1,test_y1
    test_inputs_2, test_targets_2 = test_x2,test_y2
    test_inputs_3, test_targets_3 = test_x3,test_y3

    # record loss descent
    BCE=[]


    # main nn object
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1 = torch.nn.Parameter(torch.randn(p))
            self.linear2 = torch.nn.Linear(p,10)
            self.linear3 = torch.nn.Linear(10,10)
            self.linear4 = torch.nn.Linear(10,1)

        def forward(self, x):
            layer1_out = self.linear1*x
            layer2_out = F.relu(self.linear2(layer1_out))
            layer3_out = F.relu(self.linear3(layer2_out))
            out= torch.sigmoid(self.linear4(layer3_out))
            return out, layer1_out, layer2_out,layer3_out



    model_1=MLP(seed=1)
    model_2=MLP(seed=2)
    model_3=MLP(seed=1)

    loss_fn = torch.nn.BCELoss()
    params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
    optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)

    # learning rate decay scheme
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

    loss_record=[]
    # loop for max_iteration times
    for t in range(max_iteration):

        # renew optimizer
        optimizer.zero_grad()

        # forward propagate
        out_1, layer1_out_1, layer2_out_1,layer3_out_1= model_1(inputs_1)
        out_2, layer1_out_2, layer2_out_2,layer3_out_2= model_2(inputs_2)
        out_3, layer1_out_3, layer2_out_3,layer3_out_3= model_3(inputs_3)



        # extract parameters
        #[:-1] for leaving out bias term#
        model_1_all_linear1_params = model_1.linear1
        model_1_all_linear2_params= torch.cat([x.view(-1) for x in model_1.linear2.parameters()][:-1])
        model_1_all_linear3_params= torch.cat([x.view(-1) for x in model_1.linear3.parameters()][:-1])
        model_1_all_linear4_params= torch.cat([x.view(-1) for x in model_1.linear4.parameters()][:-1])

        model_2_all_linear1_params = model_1.linear1
        model_2_all_linear2_params= torch.cat([x.view(-1) for x in model_2.linear2.parameters()][:-1])
        model_2_all_linear3_params= torch.cat([x.view(-1) for x in model_2.linear3.parameters()][:-1])
        model_2_all_linear4_params= torch.cat([x.view(-1) for x in model_2.linear4.parameters()][:-1])

        model_3_all_linear1_params = model_3.linear1
        model_3_all_linear2_params= torch.cat([x.view(-1) for x in model_3.linear2.parameters()][:-1])
        model_3_all_linear3_params= torch.cat([x.view(-1) for x in model_3.linear3.parameters()][:-1])
        model_3_all_linear4_params= torch.cat([x.view(-1) for x in model_3.linear4.parameters()][:-1])


        # compute loss

        all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)
        all_linear234_params=torch.cat((model_1_all_linear2_params,model_1_all_linear3_params, model_1_all_linear4_params,
                                         model_2_all_linear2_params,model_2_all_linear3_params, model_2_all_linear4_params,
                                         model_3_all_linear2_params,model_3_all_linear3_params,model_3_all_linear4_params),0)

        BCE_loss_1 = loss_fn(out_1, targets_1.reshape(-1,1))
        BCE_loss_2 = loss_fn(out_2, targets_2.reshape(-1,1))
        BCE_loss_3 = loss_fn(out_3, targets_3.reshape(-1,1))

        non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)

        l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))
        l2_regularization = lambda2 * torch.sum(all_linear234_params.pow(2))
        BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
        loss =BCE_loss+l1_regularization +l2_regularization

        loss_record.append(loss.item())
        # record loss
        BCE.append(BCE_loss.item())

        # compute derivative
        loss.backward()

        # gradient descent
        optimizer.step()

        # learning rate decay
        scheduler.step()
        
    
####################################   prediction ####################################      

    prediction_1= (model_1(test_x1)[0]>0.5).clone().int()
    target_1= test_y1.reshape(-1,1).int()

    prediction_2= (model_2(test_x2)[0]>0.5).clone().int()
    target_2= test_y2.reshape(-1,1).int()

    prediction_3= (model_3(test_x3)[0]>0.5).clone().int()
    target_3= test_y3.reshape(-1,1).int()

    prediction=np.append(prediction_1,prediction_2)
    prediction=np.append(prediction,prediction_3)
    prediction=prediction.tolist()

    target=np.append(target_1,target_2)
    target=np.append(target,target_3)
    target=target.tolist()

    final_matrix=confusion_matrix(target,prediction)
    TPR.append(recall_score(target,prediction))#TPR
    TNR.append(final_matrix[0,0]/(sum(final_matrix[0,:])))#TNR
    GM.append(math.sqrt(TPR[-1]*TNR[-1]))
    accuracy.append(accuracy_score(target,prediction))
    precision.append(precision_score(target,prediction))
    f1.append(f1_score(target,prediction))
    
    
####################################   variable selection ####################################     
    model_1_weight=model_1_all_linear1_params.detach().numpy().copy().reshape(1,-1)
    model_2_weight=model_1_all_linear1_params.detach().numpy().copy().reshape(1,-1)
    model_3_weight=model_1_all_linear1_params.detach().numpy().copy().reshape(1,-1)

    threshold=0.12
    mcl_w=np.zeros(p)

    for j in range(p):
        mcl_w[j]=cubic_root(model_1_weight[0,j]*model_2_weight[0,j]*model_3_weight[0,j])

    max_mcl_w=max(abs(mcl_w))
    for j in range(p):
        if (abs(mcl_w[j]))<=threshold*max_mcl_w:
            model_1_weight[0,j]=0
            model_2_weight[0,j]=0
            model_3_weight[0,j]=0
        else:
            model_1_weight[0,j]=model_1_weight[0,j]
            model_2_weight[0,j]=model_2_weight[0,j]
            model_3_weight[0,j]=model_3_weight[0,j]
    
    index=np.nonzero(model_1_weight)[1]
    
    vs[:,index]+=1
    
    beta=np.array(beta_1)
    trueindex=np.where(beta!=0)[0]
    trueindex=trueindex.tolist()
    trueindex=set(trueindex)
    
    index=np.nonzero(model_1_weight)[1]
    index=index.tolist()
    index=set(index)
    
    TP=len(trueindex.intersection(index))
    FP=len(index.difference(trueindex))

    FN=len(trueindex)-TP
    TN=p-len(trueindex)-FP
    vsSEN.append(TP/(TP+FN))
    vsSPE.append(TN/(TN+FP))
    vsGM.append(math.sqrt(vsSEN[-1]*vsSPE[-1]))
    vsMR.append((FP+FN)/(TP+FN+TN+FP))
    vsCCR.append(1-vsMR[-1])
    
    print(i)
    

2022-11-14 21:26:51,733	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2022-11-14 21:26:52,576	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,ga,lambda1,lambda2,lr,iter,total time (s),my_test_BCE_loss
train_model_00595_00000,TERMINATED,10.224.0.12:8878,0.3,0.03,0.003,0.8,1,11.503,1.46427
train_model_00595_00001,TERMINATED,10.224.0.12:8905,0.5,0.03,0.001,0.8,1,13.831,1.65102
train_model_00595_00002,TERMINATED,10.224.0.12:8907,0.4,0.05,0.001,0.6,1,12.2273,1.55931
train_model_00595_00003,TERMINATED,10.224.0.12:8909,0.4,0.05,0.002,0.8,1,12.0178,1.37716
train_model_00595_00004,TERMINATED,10.224.0.12:8911,0.4,0.04,0.002,0.7,1,11.6533,1.47972
train_model_00595_00005,TERMINATED,10.224.0.12:8912,0.4,0.03,0.002,0.8,1,12.1553,1.49049
train_model_00595_00006,TERMINATED,10.224.0.12:8913,0.4,0.04,0.002,0.7,1,12.7376,1.47972
train_model_00595_00007,TERMINATED,10.224.0.12:8916,0.4,0.03,0.001,0.7,1,11.926,1.64313
train_model_00595_00008,TERMINATED,10.224.0.12:8919,0.5,0.03,0.003,0.8,1,12.614,1.38249
train_model_00595_00009,TERMINATED,10.224.0.12:8921,0.5,0.03,0.001,0.6,1,12.9936,1.64738


(raylet) [2022-11-14 21:27:01,621 E 7959 7984] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-14_21-26-49_706836_7845 is over 95% full, available space: 509648896; capacity: 29394726912. Object creation will fail if spilling is required.


Result for train_model_00595_00000:
  date: 2022-11-14_21-27-06
  done: false
  experiment_id: dd866683cf964ea2b3a9a6e0b0eb3aa6
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.4642709493637085
  node_ip: 10.224.0.12
  pid: 8878
  time_since_restore: 11.502973794937134
  time_this_iter_s: 11.502973794937134
  time_total_s: 11.502973794937134
  timestamp: 1668432426
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 00595_00000
  warmup_time: 0.0035490989685058594
  
Result for train_model_00595_00000:
  date: 2022-11-14_21-27-06
  done: true
  experiment_id: dd866683cf964ea2b3a9a6e0b0eb3aa6
  experiment_tag: 0_ga=0.3000,lambda1=0.0300,lambda2=0.0030,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.4642709493637085
  node_ip: 10.224.0.12
  pid: 8878
  time_since_restore: 11.502973794937134
  time_this_iter_s: 11.502973794937134
  time_total_s: 11.502973794937134
  timestamp: 1668432426
  timest

Result for train_model_00595_00022:
  date: 2022-11-14_21-27-10
  done: false
  experiment_id: 7b3f556dd6ae446eb63fe7ec3fb4cfaa
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.6169432878494263
  node_ip: 10.224.0.12
  pid: 8946
  time_since_restore: 11.906601428985596
  time_this_iter_s: 11.906601428985596
  time_total_s: 11.906601428985596
  timestamp: 1668432430
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 00595_00022
  warmup_time: 0.0046863555908203125
  
Result for train_model_00595_00019:
  date: 2022-11-14_21-27-10
  done: false
  experiment_id: 20e5c80659724049abc7157521b04837
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.3380090951919557
  node_ip: 10.224.0.12
  pid: 8940
  time_since_restore: 12.156932830810547
  time_this_iter_s: 12.156932830810547
  time_total_s: 12.156932830810547
  timestamp: 1668432430
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 00595_00019


...

Result for train_model_2_7085d_00210:
  date: 2022-11-14_21-31-54
  done: false
  experiment_id: 5fb3722d31fe4142979f4a8ea7bd325b
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.9464142445203688
  node_ip: 10.224.0.12
  pid: 13806
  time_since_restore: 14.681645393371582
  time_this_iter_s: 14.681645393371582
  time_total_s: 14.681645393371582
  timestamp: 1668432714
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 7085d_00210
  warmup_time: 0.007678031921386719
  
Result for train_model_2_7085d_00210:
  date: 2022-11-14_21-31-54
  done: true
  experiment_id: 5fb3722d31fe4142979f4a8ea7bd325b
  experiment_tag: 210_eta=0.4000,lambda1=0.0400,lambda2=0.0020,lr=0.7000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.9464142445203688
  node_ip: 10.224.0.12
  pid: 13806
  time_since_restore: 14.681645393371582
  time_this_iter_s: 14.681645393371582
  time_total_s: 14.681645393371582
  timestamp: 1668432714
  

Result for train_model_2_7085d_00216:
  date: 2022-11-14_21-31-56
  done: false
  experiment_id: 3f06f34a5153473b9ed7df605ff43a49
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.9514250212333606
  node_ip: 10.224.0.12
  pid: 13820
  time_since_restore: 15.51163387298584
  time_this_iter_s: 15.51163387298584
  time_total_s: 15.51163387298584
  timestamp: 1668432716
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 7085d_00216
  warmup_time: 0.006546497344970703
  
Result for train_model_2_7085d_00221:
  date: 2022-11-14_21-31-56
  done: false
  experiment_id: 11308726d17e4391bb5a2975cd2d6513
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.9541170014261997
  node_ip: 10.224.0.12
  pid: 13770
  time_since_restore: 15.037707567214966
  time_this_iter_s: 15.037707567214966
  time_total_s: 15.037707567214966
  timestamp: 1668432716
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 7085d_00221


2022-11-14 21:31:58,523	WARNING tune.py:686 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Result for train_model_2_7085d_00219:
  date: 2022-11-14_21-31-58
  done: false
  experiment_id: 1aa71e5f3c0e4f1aaf66a6090924342a
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.9456821770394607
  node_ip: 10.224.0.12
  pid: 13816
  time_since_restore: 17.0430166721344
  time_this_iter_s: 17.0430166721344
  time_total_s: 17.0430166721344
  timestamp: 1668432718
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 7085d_00219
  warmup_time: 0.00813603401184082
  


(raylet) [2022-11-14 21:32:01,959 E 7959 7984] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-14_21-26-49_706836_7845 is over 95% full, available space: 491810816; capacity: 29394726912. Object creation will fail if spilling is required.


In [8]:
####################################   prediction ####################################   
from numpy import *
mean(TPR)
mean(TNR)
mean(GM)
mean(accuracy)
mean(precision)
mean(f1)


0.8638232235600656

0.8763424678749756

0.8696485241227065

0.8691666666666666

0.8787263520342238

0.8702829716355073

(raylet) [2022-11-14 20:56:50,713 E 25215 25233] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-14_20-23-57_013792_1834 is over 95% full, available space: 280997888; capacity: 29394726912. Object creation will fail if spilling is required.


In [9]:
####################################   variable selection ####################################   
mean(vsSEN)
mean(vsSPE)
mean(vsGM)
mean(vsMR)
mean(vsCCR)

0.93

0.9822222222222223

0.9548247313928107

0.023

0.977

(raylet) [2022-11-14 20:57:00,723 E 25215 25233] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-14_20-23-57_013792_1834 is over 95% full, available space: 280952832; capacity: 29394726912. Object creation will fail if spilling is required.


In [10]:
vs

array([[ 9., 10., 10., 10., 10.,  5., 10.,  9., 10., 10.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,
         0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,
         0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

(raylet) [2022-11-14 20:57:10,733 E 25215 25233] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-14_20-23-57_013792_1834 is over 95% full, available space: 280899584; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-14 20:57:20,744 E 25215 25233] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-14_20-23-57_013792_1834 is over 95% full, available space: 280883200; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-14 20:57:30,755 E 25215 25233] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-14_20-23-57_013792_1834 is over 95% full, available space: 280870912; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-14 20:57:40,765 E 25215 25233] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-14_20-23-57_013792_1834 is over 95% full, available space: 280854528; capacity: 29394726912. Object creation will fail if sp

In [31]:
i

9

(raylet) [2022-10-21 15:55:01,059 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 845570048; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 15:55:11,066 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 845410304; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 15:55:21,075 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 845295616; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 15:55:31,082 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 845205504; capacity: 29394726912. Object creation will fail i

(raylet) [2022-10-21 16:00:11,291 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 841715712; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 16:00:21,298 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 841609216; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 16:00:31,306 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 841502720; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 16:00:41,313 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 841404416; capacity: 29394726912. Object creation will fail i

(raylet) [2022-10-21 16:05:21,521 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 839045120; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 16:05:31,529 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 839032832; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 16:05:41,537 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 838963200; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 16:05:51,543 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 838811648; capacity: 29394726912. Object creation will fail i

(raylet) [2022-10-21 16:10:31,739 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 836837376; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 16:10:41,745 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 838131712; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 16:10:51,753 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 836706304; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 16:11:01,759 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 836591616; capacity: 29394726912. Object creation will fail i

(raylet) [2022-10-21 16:15:42,171 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 834625536; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 16:15:52,187 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 834441216; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 16:16:02,218 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 834437120; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 16:16:12,233 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 835702784; capacity: 29394726912. Object creation will fail i

(raylet) [2022-10-21 16:20:52,493 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 832335872; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 16:21:02,500 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 832286720; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 16:21:12,508 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 832270336; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 16:21:22,515 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 832196608; capacity: 29394726912. Object creation will fail i

(raylet) [2022-10-21 16:26:02,716 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 830185472; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 16:26:12,723 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 831078400; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 16:26:22,729 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 830029824; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-21 16:26:32,735 E 27159 27177] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-20_16-00-38_385267_26810 is over 95% full, available space: 829960192; capacity: 29394726912. Object creation will fail i